# LangGraph MCP Agent Implementation

## Overview

In this exercise, you'll build a sophisticated AI agent that combines **LangGraph** (for agent orchestration) with **Model Context Protocol (MCP)** (for external tool integration). This represents an advanced pattern in AI agent development where agents can dynamically discover and use external tools and services.

## Learning Objectives

By the end of this exercise, you will be able to:

1. **Understand MCP Architecture**: Learn how Model Context Protocol enables secure integration with external tools
2. **Build HTTP-based Tool Clients**: Create LangChain tools that communicate with MCP servers via HTTP
3. **Implement ReAct Agents**: Use LangGraph to create reasoning and acting agents
4. **Handle Complex Workflows**: Process multi-step queries requiring multiple tool calls
5. **Manage Server Lifecycles**: Start, monitor, and stop external MCP servers

## Key Concepts

### Model Context Protocol (MCP)
- **Purpose**: Standardized way for AI applications to integrate with external tools and data sources
- **Security**: Provides secure, controlled access to external capabilities
- **Flexibility**: Allows runtime discovery of available tools and their capabilities

### LangGraph ReAct Pattern
- **Reasoning**: Agent analyzes the query and determines what tools are needed
- **Acting**: Agent executes tools to gather information or perform actions
- **Iteration**: Process repeats until the query is fully resolved

### HTTP vs. Local Tool Integration
- **HTTP Tools**: Connect to separately hosted MCP servers (more realistic for production)
- **Local Tools**: Embed tool logic directly in the notebook (simpler for development)

## Architecture Overview

```
┌─────────────────┐    HTTP     ┌─────────────────┐
│   LangGraph     │◄──────────► │   MCP Server    │
│     Agent       │   Requests  │  (Python App)   │
│                 │             │                 │
│ ┌─────────────┐ │             │ ┌─────────────┐ │
│ │LangChain    │ │             │ │ Math Tools  │ │
│ │HTTP Tools   │ │             │ │ Text Tools  │ │
│ └─────────────┘ │             │ │ Data Tools  │ │
└─────────────────┘             │ └─────────────┘ │
                                └─────────────────┘
```

## Step 1: Import Dependencies and Setup Environment

First, let's import all the necessary libraries for our MCP agent implementation.

### Instructions:
1. Import all required libraries for HTTP communication, LangChain, and LangGraph
2. Import environment management libraries for secure credential handling
3. Import async libraries for handling concurrent operations

### TODO: Complete the import statements below

In [ ]:
# TODO: Import standard libraries
# Hint: You'll need asyncio, json, sys, subprocess, time, pathlib, os, getpass
import # Complete the imports

# TODO: Import HTTP client libraries
# Hint: Import requests for HTTP communication

# TODO: Import LangChain and LangGraph components
# Hint: You'll need AzureChatOpenAI, tool decorator, message types, create_react_agent, MemorySaver
from langchain_openai import # Complete the import
from langchain_core.tools import # Complete the import
from langchain_core.messages import # Complete the imports
from langgraph.prebuilt import # Complete the import
from langgraph.checkpoint.memory import # Complete the import

# TODO: Import environment setup
from dotenv import # Complete the import

print("All libraries imported successfully!")

## Step 2: Environment Variable Configuration

Configure environment variables for secure API key management.

### Instructions:
1. Load environment variables from a .env file if present
2. Prompt for Azure OpenAI API key if not already configured
3. Ensure secure credential handling

### TODO: Complete the environment configuration

In [ ]:
# TODO: Load environment variables from .env file
# Hint: Use load_dotenv() function

# TODO: Check if API key exists, if not prompt for it
# Hint: Use os.environ.get() to check and getpass.getpass() to prompt securely
if not os.environ.get("AZURE_OPENAI_API_KEY"):
    os.environ["AZURE_OPENAI_API_KEY"] = # Complete this line

print("API key configured successfully!")

## Step 3: Azure OpenAI LLM Initialization

Initialize the Azure OpenAI chat model with optimized parameters for tool usage.

### Instructions:
1. Create an AzureChatOpenAI instance with the provided endpoint
2. Use GPT-4o model with low temperature for consistent tool usage
3. Configure API version and other parameters

### TODO: Initialize the Azure OpenAI model

In [ ]:
# TODO: Initialize Azure OpenAI Chat model
# Hint: Use the endpoint "https://aoi-ext-eus-aiml-profx-01.openai.azure.com/"
# Hint: Use model "gpt-4o", api_version "2024-12-01-preview", temperature 0.1
llm = AzureChatOpenAI(
    azure_endpoint=# Complete this parameter
    api_key=# Complete this parameter
    model=# Complete this parameter
    api_version=# Complete this parameter
    temperature=# Complete this parameter (use 0.1 for consistent tool usage)
)

print("Azure OpenAI LLM initialized successfully!")
print(f"Model: gpt-4o")
print(f"Temperature: 0.1 (optimized for tool usage)")

## Step 4: MCP Server Configuration and Health Check

Configure the connection to the MCP HTTP server and implement health monitoring.

### Instructions:
1. Define the MCP server base URL
2. Implement a health check function to verify server availability
3. Handle connection errors gracefully

### TODO: Complete the MCP server configuration

In [ ]:
# TODO: Define MCP server configuration
MCP_SERVER_BASE_URL = # Set to "http://localhost:8000"

def check_mcp_server() -> bool:
    """Check if the MCP HTTP server is running and accessible."""
    try:
        # TODO: Make a GET request to the health endpoint
        # Hint: Use requests.get() with URL "{MCP_SERVER_BASE_URL}/health" and timeout=5
        response = requests.get(# Complete this line
        
        # TODO: Check if response status code is 200
        if response.status_code == 200:
            return True
        else:
            print(f"Server responded with status code: {response.status_code}")
            return False
    except requests.exceptions.RequestException as e:
        print(f"Server not accessible: {e}")
        return False

print("MCP server configuration ready!")
print(f"Server URL: {MCP_SERVER_BASE_URL}")

## Step 5: HTTP MCP Client Tools - Mathematical Operations

Create LangChain tools that communicate with the MCP server for mathematical operations.

### Instructions:
1. Use the `@tool` decorator to create LangChain tools
2. Implement HTTP POST requests to the MCP server endpoints
3. Handle responses and errors appropriately
4. Format results for the agent

### TODO: Complete the mathematical tools implementation

In [ ]:
# Import the tool decorator
from langchain_core.tools import tool

@tool
def http_math_statistics(values: list) -> str:
    """
    Calculate comprehensive statistics (mean, median, std dev, etc.) for a dataset.
    
    Args:
        values: List of numerical values
        
    Returns:
        Comprehensive statistical analysis of the dataset
    """
    try:
        # TODO: Make POST request to MCP server math endpoint
        # Hint: Use requests.post() with URL "{MCP_SERVER_BASE_URL}/math"
        # Hint: Send JSON payload with operation="statistics" and values=values
        response = requests.post(
            # Complete the URL
            json={
                "operation": # Complete this
                "values": # Complete this
            },
            timeout=10
        )
        
        # TODO: Raise exception for bad status codes
        response.raise_for_status()
        result = response.json()
        
        if result["success"]:
            stats = result["result"]
            # TODO: Format the statistics nicely
            # Hint: Return a formatted string with all statistical measures
            return f"""Statistical Analysis:
                        - Count: {stats['count']}
                        - Mean: {stats['mean']:.4f}
                        - Median: {stats['median']:.4f}
                        - Standard Deviation: {stats['std_dev']:.4f}
                        - Variance: {stats['variance']:.4f}
                        - Min: {stats['min']}
                        - Max: {stats['max']}
                        - Range: {stats['range']}
                        - Sum: {stats['sum']}"""
        else:
            return f"Error: {result.get('message', 'Unknown error')}"
    except Exception as e:
        return f"Error calling math statistics: {str(e)}"

@tool
def http_solve_quadratic(a: float, b: float, c: float) -> str:
    """
    Solve a quadratic equation ax² + bx + c = 0.
    
    Args:
        a: Coefficient of x²
        b: Coefficient of x
        c: Constant term
        
    Returns:
        Solution(s) to the quadratic equation
    """
    try:
        # TODO: Implement quadratic solver HTTP call
        # Hint: Similar to statistics but with operation="quadratic" and a, b, c parameters
        response = requests.post(
            f"{MCP_SERVER_BASE_URL}/math",
            json={
                # Complete the JSON payload for quadratic operation
            },
            timeout=10
        )
        response.raise_for_status()
        result = response.json()
        
        if result["success"]:
            quad_result = result["result"]
            # TODO: Extract and format quadratic results
            # Hint: Get roots, root_type, and discriminant from quad_result
            return f"""Quadratic Equation {a}x² + {b}x + {c} = 0:
                        - Discriminant: {quad_result['discriminant']}
                        - Root Type: {quad_result['root_type']}
                        - Solutions: {quad_result['roots']}"""
        else:
            return f"Error: {result.get('message', 'Unknown error')}"
    except Exception as e:
        return f"Error solving quadratic: {str(e)}"

print("Mathematical tools implemented successfully!")

## Step 6: HTTP MCP Client Tools - Text Operations

Create LangChain tools for text analysis and manipulation.

### Instructions:
1. Implement text analysis tool for comprehensive text statistics
2. Create information extraction tool for emails, phone numbers, URLs, etc.
3. Add text transformation capabilities

### TODO: Complete the text processing tools

In [ ]:
@tool
def http_text_analysis(text: str) -> str:
    """
    Perform comprehensive text analysis including character count, word count, etc.
    
    Args:
        text: The text to analyze
        
    Returns:
        Detailed analysis of the text
    """
    try:
        # TODO: Make POST request to text analysis endpoint
        # Hint: Use "{MCP_SERVER_BASE_URL}/text" with operation="analyze"
        response = requests.post(
            # Complete the implementation
        )
        response.raise_for_status()
        result = response.json()
        
        if result["success"]:
            return f"Text Analysis Results:\n{result['result']}"
        else:
            return f"Error: {result.get('message', 'Unknown error')}"
    except Exception as e:
        return f"Error analyzing text: {str(e)}"

@tool
def http_extract_information(text: str, extraction_type: str) -> str:
    """
    Extract specific information from text (emails, phone numbers, URLs, etc.).
    
    Args:
        text: The text to extract information from
        extraction_type: Type of information to extract (emails, phone_numbers, urls, dates, numbers)
        
    Returns:
        Extracted information of the specified type
    """
    try:
        # TODO: Implement information extraction HTTP call
        # Hint: Use operation="extract" with text and extraction_type parameters
        response = requests.post(
            f"{MCP_SERVER_BASE_URL}/text",
            json={
                # Complete the JSON payload
            },
            timeout=10
        )
        response.raise_for_status()
        result = response.json()
        
        if result["success"]:
            return f"Extracted {extraction_type}:\n{result['result']}"
        else:
            return f"Error: {result.get('message', 'Unknown error')}"
    except Exception as e:
        return f"Error extracting {extraction_type}: {str(e)}"

@tool
def http_transform_text(text: str, transformation: str) -> str:
    """
    Transform text using various transformations (uppercase, lowercase, title_case, pig_latin).
    
    Args:
        text: The text to transform
        transformation: Type of transformation (uppercase, lowercase, title_case, pig_latin)
        
    Returns:
        Transformed text
    """
    try:
        # TODO: Implement text transformation HTTP call
        # Hint: Use operation="transform" with extraction_type parameter for transformation type
        response = requests.post(
            # Complete the implementation
        )
        response.raise_for_status()
        result = response.json()
        
        if result["success"]:
            return f"Transformed text ({transformation}):\n{result['result']}"
        else:
            return f"Error: {result.get('message', 'Unknown error')}"
    except Exception as e:
        return f"Error transforming text: {str(e)}"

print("Text processing tools implemented successfully!")

## Step 7: Tool Collection and Agent Creation

Collect all tools and create the LangGraph ReAct agent.

### Instructions:
1. Collect all HTTP MCP tools into a list
2. Create an async function to build the ReAct agent
3. Implement server health checking before agent creation
4. Configure memory for conversation persistence

### TODO: Complete the agent creation implementation

In [ ]:
# TODO: Collect all HTTP MCP tools into a list
# Hint: Create a list with all the tools you've defined above
http_mcp_tools = [
    # Add all your tools here
]

async def create_http_mcp_agent():
    """
    Create a LangGraph ReAct agent using HTTP-based MCP tools.
    
    Returns:
        Configured agent ready for use
    """
    # TODO: Check if MCP server is running
    if not check_mcp_server():
        print("ERROR: MCP HTTP server is not running!")
        print("Please start it with: python ../../MCP/mcp-http-server.py")
        return None
    
    print("MCP HTTP server is accessible")
    
    # TODO: Initialize memory for conversation persistence
    # Hint: Use MemorySaver() from langgraph.checkpoint.memory
    memory = # Complete this line
    
    # TODO: Create the ReAct agent with HTTP-based MCP tools
    # Hint: Use create_react_agent with model=llm, tools=http_mcp_tools, checkpointer=memory
    agent = create_react_agent(
        model=# Complete this parameter
        tools=# Complete this parameter
        checkpointer=# Complete this parameter
    )
    
    print("HTTP-based MCP Agent created successfully!")
    print(f"Configured with {len(http_mcp_tools)} tools")
    print("Connected to separately hosted MCP server")
    
    return agent

print("Agent creation function ready!")
print(f"Available tools: {len(http_mcp_tools)}")

## Step 8: Agent Interaction and Streaming

Implement the streaming interface for real-time agent interactions.

### Instructions:
1. Create an async function to handle user queries
2. Implement streaming to show agent progress in real-time
3. Handle conversation threading for context persistence
4. Process agent responses and tool calls

### TODO: Complete the streaming implementation

In [ ]:
async def stream_http_agent_updates(user_input: str):
    """
    Stream updates from the HTTP-based MCP agent.
    
    Args:
        user_input (str): The input from the user.
    """
    # TODO: Create the agent
    agent = await create_http_mcp_agent()
    if agent is None:
        return
    
    # TODO: Configuration for conversation threading
    # Hint: Use a dictionary with "configurable" and "thread_id" keys
    config = {"configurable": {"thread_id": # Give it a unique thread ID}}
    
    print(f"\nProcessing query with HTTP MCP Agent...")
    print(f"Query: {user_input}")
    print("\n" + "="*60)
    
    try:
        # TODO: Stream the agent's response
        # Hint: Use agent.astream() with messages containing HumanMessage(content=user_input)
        async for event in agent.astream(
            {"messages": [# Create a HumanMessage with the user input]},
            config=config
        ):
            try:
                for value in event.values():
                    if "messages" in value and value["messages"]:
                        last_message = value["messages"][-1]
                        if hasattr(last_message, 'content') and last_message.content:
                            print("Assistant:", last_message.content)
                            print("-" * 40)
            except Exception as e:
                print(f"WARNING: Error processing event: {e}")
                
    except Exception as e:
        print(f"ERROR: Error with HTTP MCP agent: {e}")
        
    print("\nHTTP MCP Agent response completed!")

print("Streaming function ready!")
print("Ready to process queries with HTTP communication")

## Step 9: MCP Server Management (Optional)

This section provides tools to automatically start and stop the MCP server. This is optional as the server can be started manually.

### Instructions:
1. Implement server startup functionality
2. Add health monitoring during startup
3. Handle server shutdown gracefully

### TODO: Complete the server management functions (Optional)

In [ ]:
import subprocess
import time
from pathlib import Path

# Global variable to track server process
http_server_process = None

def start_http_mcp_server():
    """Start the HTTP MCP server in the background."""
    global http_server_process
    
    print("Starting HTTP MCP server...")
    
    # TODO: Define paths to the MCP server
    # Hint: Use Path("../../MCP").resolve() and server_script = mcp_dir / "mcp-http-server.py"
    mcp_dir = Path(# Complete this).resolve()
    server_script = mcp_dir / # Complete this
    
    if not server_script.exists():
        print(f"ERROR: Server script not found: {server_script}")
        return False
    
    # Check if server is already running
    if check_mcp_server():
        print("SUCCESS: MCP server is already running!")
        return True
    
    try:
        # TODO: Start the server in background using subprocess.Popen
        # Hint: Use [sys.executable, str(server_script)] as the command
        http_server_process = subprocess.Popen(
            # Complete the command list
            cwd=mcp_dir,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        
        print(f"INFO: Server starting with PID: {http_server_process.pid}")
        
        # Wait for server to initialize
        for i in range(15):  # Wait up to 15 seconds
            time.sleep(1)
            print(f"INFO: Waiting for server... ({i+1}/15)")
            
            # Check if process failed
            if http_server_process.poll() is not None:
                stdout, stderr = http_server_process.communicate()
                print(f"ERROR: Server process failed!")
                print(f"stdout: {stdout}")
                print(f"stderr: {stderr}")
                return False
            
            # Check if server is responding
            if check_mcp_server():
                print("SUCCESS: HTTP MCP server started successfully!")
                print(f"INFO: Server available at: {MCP_SERVER_BASE_URL}")
                return True
        
        print("WARNING: Server started but not responding to health checks")
        return False
        
    except Exception as e:
        print(f"ERROR: Failed to start server: {e}")
        return False

def stop_http_mcp_server():
    """Stop the HTTP MCP server."""
    global http_server_process
    
    if http_server_process and http_server_process.poll() is None:
        print("INFO: Stopping HTTP MCP server...")
        http_server_process.terminate()
        
        try:
            http_server_process.wait(timeout=5)
            print("SUCCESS: Server stopped gracefully")
        except subprocess.TimeoutExpired:
            http_server_process.kill()
            print("INFO: Server force stopped")
        
        http_server_process = None
    else:
        print("INFO: No server process to stop")

print("Server management functions ready!")
print("Use start_http_mcp_server() to start the server if needed")

## Step 10: Test Mathematical Operations

Test the agent's mathematical capabilities with statistical analysis and equation solving.

### Instructions:
1. Define a test query that requires multiple mathematical operations
2. Run the query through the agent
3. Observe how the agent uses multiple tools to complete the task

### TODO: Test the mathematical operations

In [ ]:
# TODO: Define a comprehensive mathematical query
# This query should test multiple mathematical tools
math_query = """
I have a dataset with the following values: [85, 92, 78, 96, 89, 91, 87, 94, 82, 88].
Please calculate the mean, median, and standard deviation for this dataset.
Also, solve the quadratic equation 2x² - 7x + 3 = 0.
Finally, analyze the distribution characteristics of the data.
"""

print("Testing Mathematical Operations via HTTP MCP Server")
print("=" * 60)
print(f"Query: {math_query}")
print("\nAgent Response (via HTTP MCP Server):")
print("=" * 60)

# TODO: Run the query using the HTTP-based MCP agent
# Hint: Use await stream_http_agent_updates(math_query)
await # Complete this line

## Step 11: Test Text Processing Operations

Test the agent's text analysis and information extraction capabilities.

### Instructions:
1. Create a test query with text that contains extractable information
2. Request multiple text processing operations
3. Observe how the agent handles multi-step text processing

### TODO: Test the text processing operations

In [ ]:
# TODO: Define a comprehensive text processing query
text_query = """
Please analyze the following text and extract useful information:

"Artificial Intelligence and Machine Learning are revolutionizing technology. 
Our research team has achieved 95.3% accuracy in natural language processing tasks.
Contact us at research@aicompany.com or call (555) 123-4567 for collaboration.
Visit our website at https://ai-revolution.com to learn more about our work!"

I need:
1. A comprehensive text analysis with word count, character analysis, etc.
2. Extract all email addresses and phone numbers from the text
3. Transform the first sentence to pig latin
4. Provide insights about the text structure and content
"""

print("\n" + "="*60)
print("Testing Text Processing via HTTP MCP Server")
print("=" * 60)
print(f"Query: {text_query}")
print("\nAgent Response (via HTTP MCP Server):")
print("=" * 60)

# TODO: Run the text processing query
# Hint: Use await stream_http_agent_updates(text_query)
await # Complete this line

## Congratulations! 🎉

You have successfully implemented a LangGraph ReAct agent with Model Context Protocol (MCP) integration!

## What You've Accomplished

1. **Built HTTP-based Tool Integration**: Created LangChain tools that communicate with external MCP servers
2. **Implemented ReAct Pattern**: Used LangGraph to create an agent that reasons and acts iteratively
3. **Managed External Services**: Handled server lifecycle and health monitoring
4. **Processed Complex Queries**: Demonstrated multi-step reasoning with mathematical and text processing tasks
5. **Achieved Modularity**: Separated tool logic from agent logic for maintainable architecture

## Key Takeaways

- **MCP provides standardized tool integration** for AI agents
- **HTTP-based tools enable distributed architectures** where tools run as separate services
- **LangGraph's ReAct pattern enables complex reasoning** through iterative tool usage
- **Async streaming provides real-time feedback** during agent execution
- **Proper error handling is crucial** for robust agent systems

## Next Steps

Try these advanced exercises:

1. **Add Custom Tools**: Extend the MCP server with your own tool endpoints
2. **Implement Caching**: Add response caching to improve performance
3. **Add Authentication**: Secure the MCP server with API keys or tokens
4. **Create Tool Chains**: Build tools that depend on outputs from other tools
5. **Monitor Performance**: Add logging and metrics to track agent performance

## Clean Up

Don't forget to stop the MCP server when you're done:

```python
stop_http_mcp_server()
```

Great job completing this advanced AI agent exercise! 🚀